In [1]:
#countvectorizer-LDA and calculation of perplexity #module 5.0 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV



df1 = pd.read_csv("../Files/Figure3/Table7.csv")
df1["period"]=1
df2 = pd.read_csv("../Files/Figure3/Table8.csv")
df2["period"]=2
df3 = pd.read_csv("../Files/Figure3/Table9.csv")
df3["period"]=3
df4 = pd.read_csv("../Files/Figure3/Table10.csv")
df4["period"]=4
df = pd.concat([df1,df2,df3,df4],axis=0)
df.drop_duplicates(subset="PMID",inplace=True)
df.reset_index(inplace=True,drop=True)
display(df)


nlp = spacy.load("en_core_sci_lg")
with open("../Files/SupplementalTables/SupplementalTable1.txt", "r", encoding='utf-8') as f:
    stopwords = [w.strip() for w in f]
print("=====\n stopwords\n")
print(stopwords)

dic_corpus={}
for raw in df[["PMID","abstract"]].itertuples():
    text =""
    text_cleaned=""
    doc = nlp(str(raw[2]))
    for ent in doc.ents:
        text = text+" "+str(ent).lower()
        ls = text.split()
    for i in ls:
        if i not in stopwords:
            text_cleaned = text_cleaned+" "+i
    print(raw[1])
    print(text_cleaned)
    dic_corpus[raw[1]]=text_cleaned


#1 countvectorizer
corpus = [text_cleaned for text_cleaned in dic_corpus.values()]
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(corpus)
count_name=vectorizer.get_feature_names_out()

count_vec= count.toarray()
count_pd = pd.DataFrame(count_vec,index = dic_corpus.keys(),columns=count_name)


#2 LDA and n_topic optimization
perplexity = {}
for i in range(10,21):
    lda = LatentDirichletAllocation(n_components=i,random_state=16)#lda
    lda.fit(count_vec)
    perp = lda.perplexity(count_vec)
    perplexity[i]=perp
    print("topic:{}, perplexity:{}".format(i,perp))
    lda_value = lda.fit_transform(count_vec)#probability of each topic
    lda_index = np.argmax(lda_value,axis=1)#maximum of the probability of each group = component
    df_lda_index = pd.DataFrame(lda_index,index=dic_corpus.keys(),columns =["topic"])#dic_corpus.keys()=PMID
    df_period = df[["PMID","period"]].set_index(["PMID"])
    df_lda = pd.concat([df_lda_index,df_period],axis=1)
    display(df_lda)
    df_lda.to_csv("../Files/Figure4/PMID_topic/Table{}.csv".format(i+13))
    
    print("topic words top-10")
    topic_words = {}
    weight_words = {}
    n_top_words = 10
    for topic, comp in enumerate(lda.components_):
        word_idx = np.argsort(comp)[::-1][:n_top_words]
        topic_words[topic] = [count_name[i] for i in word_idx]
        weight_words[topic] = np.sort(comp)[::-1][:n_top_words]
    for topic, words in topic_words.items():
        print('Topic: %d' % topic)
        print('  %s' % ', '.join(words))
    
    #period-component
    dic_period = {}
    for j in range(i):
        ls_component = []
        for period in range(1,5):
            ls_component.append(len(df_lda[(df_lda["period"]==period)&(df_lda["topic"]==j)]))
        dic_period[j]=ls_component

    df_period = pd.DataFrame(dic_period.values(),index = dic_period.keys(),columns=["period I","period II","period III","period IV"])
    df_topic = pd.DataFrame(topic_words.values(),index = topic_words.keys(),columns=["word{}".format(i) for i in range(n_top_words)])
    df_weight = pd.DataFrame(weight_words.values(),index = weight_words.keys(),columns=["weight{}".format(i)for i in range(n_top_words)])
    df_period_topic = pd.concat([df_topic,df_weight,df_period],axis=1)
    df_period_topic.to_csv("../Files/Figure4/Topic_word/Table{}.csv".format(i+24))    
df_perplexity = pd.DataFrame(perplexity.values(),index=perplexity.keys())
df_perplexity.to_csv("../Files/Figure4/Table45.csv") 

: 

: 